<h1 style = "color:SteelBlue"> RFR Realized Rates Sample </h1>


<h2>Overview:</h2>
    <p>This template calculates realized rates for key RFR benchmarks for standard tenors (ON, 1W, 1M, 3M, 6M, 9M, 1Y).</p>
    The supported RFR benchmarks are SONIA, SOFR, €STR - in the future, TONAR, SORA - in the future and SARON.
    he rates are calculated back from most recent published fixing data from the relevant source. Model is currently using vanilla IRS swaps
    for each realized period. The swaps are sent to the financial-instrument end-point to compute 
    the accrued of each swap over the realized rate period. The realized rates are finally computed by
    annualizing the bond accrued.
    Apart from the realized rates the model supports lagged realized rates using the 'Lookback' parameter.
    User may apply 0 - 9 days lag
    in the User Interface we are presenting two separate views: Standard Periods & Custom Periods</p>
    


<h2> Standard Period functionalities:</h2>
<p> In Standard Periods user is able to get realized rates and lagged realized rates over the standard periods for a given index 
    The periods can be modified in the spreadsheet_column_info.tenor_list</p>
<ul>
<li> changing the end date -  will recompute the start and end date for each tenor all calculations get updated automatically
<li> changing the lag - will recompute LaggedRealizedRate column accordingly
<li> changing the spread - spread is applied to the accrued for each tenor updating the RealizedRate and LaggedRealizedRate
</ul>

<h2> Custom Period functionalities:</h2>

<p> In Custom Periods user is able to get realized rates and realized amount over the standard periods for a given index
    The periods can be modified in the spreadsheet_column_info.tenor_list </p>

<ul>
<li> changing the start & end dates for each row -  for each tenor all calculations get updated automatically
<li> changing the lag - will recompute RealizedRate and RealizedAmount column accordingly
<li> changing the spread - spread is applied to the accrued for each custom period, finally updating the RealizedRate
<li> changing the year basis - recomputes the year fraction
<li> changing the rounding - applies the rounding to RealizedRates and RealizedAmount

</ul>

<p>------------------------------------------------------------------------------------------------------------------------------------------------</p>
<p> This App uses Instrument Pricing Analytics (IPA) via the content layer  and Dates&Calendars via the access layer of Refinitiv Data Libraries
</ul>
<p>------------------------------------------------------------------------------------------------------------------------------------------------</p>

##### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

This notebook demonstrates how to interest rate Swap using the Instrument Pricing Analytics (IPA).

<h2> Opening a session </h2>

In [1]:
import refinitiv.data as rd
from refinitiv.data.content.ipa.financial_contracts import swap
import logging
import ipywidgets as widgets
import ipysheet as ipysheet
import pandas as pd
import time
import datetime as dt
import asyncio
import threading
from ipywidgets import *
from datetime import datetime, date
from traitlets import directional_link
from IPython.display import display, HTML
from ipysheet import *
from typing import Callable, Any
import nest_asyncio
nest_asyncio.apply()

# Application variables

log_level = logging.INFO

# Logging setup

logger = logging.getLogger()
fhandler = logging.FileHandler(filename='/home/jovyan/RFR_Realized_Rate.log', mode='a')

formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(level=log_level)


rd.open_session()

<refinitiv.data.session.Definition object at 0x1589a9eb0 {name='workspace'}>

<h2>Metadata</h2>

<h2> The metadata object contains:</h2>
<ul>
<li> the market conventions of each supported term rate (term_rate_assumption)
<li> the description of other calculation parameters (spreadsheet_column_info)
<li> additional dictionary with default data
<li> global variables
</ul>


In [2]:
metadata = {
    "term_rate_assumption":
    {
        "SONIA": {
            "currency": "GBP",
            "calendar": "UKG_FI",
            "ric": "SONIAOSR = ",
            "short_daycont": "A5",
            "daycount": "Dcb_Actual_365",
            "year_basis": "365",
            "rounding": "4",
            "compounding": "True"
        },
        "SOFR": {
            "currency": "USD",
            "calendar": "USA_FI",
            "ric": "USDSOFR = ",
            "short_daycont": "A5",
            "daycount": "Dcb_Actual_360",
            "year_basis": "360",
            "rounding": "4",
            "compounding": "False"
        },
        "TONAR": {
            "currency": "JPY",
            "calendar": "JAP_FI",
            "ric": "JPONMU = RR",
            "short_daycont": "A5",
            "daycount": "Dcb_Actual_365",
            "year_basis": "365",
            "rounding": "4",
            "compounding": "False"
        },
        "SARON": {
            "currency": "CHF",
            "calendar": "SWI_FI",
            "ric": "SARON.S",
            "short_daycont": "A0",
            "daycount": "Dcb_Actual_360",
            "year_basis": "360",
            "rounding": "4",
            "compounding": "False"
        }
    },
    "spreadsheet_column_info":
    {
        "amount": {
            'header': 'Amount',
            'type': 'int',
            'default': "1000"
        },
        "lag": {
            'header': 'Lag',
            'type': 'int',
            'default': "0"
        },
        "tenor_list": {
            'header': 'Period',
            'type': 'list',
            'default': ["1D", "1W", "1M", "2M", "3M", "6M", "9M", "1Y"]
        },
        "spread": {
            'header': 'Spread',
            'type': 'float',
            'default': "0"
        },
        "rounding": {
            'header': 'Rounding Decimals',
            'type': 'int',
            'default': "8"
        }
    }
}


# -------------------------------------------------------------------------------------------------
# create additional dictionary for default data and term rate assumptions
# we will use it later as global dictionaries.
# -------------------------------------------------------------------------------------------------

rows_custom = dict()

rows_custom_ui = []

column_info = metadata['spreadsheet_column_info']
term_rate_assumption = metadata['term_rate_assumption']

default_data = dict(periods=metadata['spreadsheet_column_info']['tenor_list']['default'],
                    lag=metadata['spreadsheet_column_info']['lag']['default'],
                    spread=metadata['spreadsheet_column_info']['spread']['default'],
                    amount=metadata['spreadsheet_column_info']['amount']['default'],
                    rounding=metadata['spreadsheet_column_info']['rounding']['default'],
                    date=datetime.now().date(),
                    index='SONIA',
                    day_count='Dcb_Actual_365',
                    lags=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                    standard_columns=['Tenor', 'Start', 'End', 'AccruedDays', 'AccruedPercent',
                                      'YearFraction', 'RealizedRate', 'BackwardShift', 'LaggedRealizedRate'],
                    custom_columns=['Tenor', 'Start', 'End', 'AccruedDays',
                                    'AccruedPercent', 'YearFraction', 'RealizedRate', 'RealizedAmount'],
                    year_basis=[360, 365],
                    )

term_rate_assumption = metadata['term_rate_assumption']

interrupted = True

<h2> Utilities section - all helper functions are here </h2>

In [3]:
# HTML(filename = r"./styles.html")

def work(progress):
    # -------------------------------------------------------------
    # worker displays progress
    # --------------------------------------------------------------
    global interrupted
    progress.value = 0.0
    progress.layout.visibility = 'visible'
    while True:
        time.sleep(0.2)
        progress.value += 0.1
        if progress.value == 1.0:
            progress.value = 0.0
        if interrupted:
            progress.value = 1.0
            progress.layout.visibility = 'hidden'
            break


# ---------------------------------------------------------------------------------------------
# Wrapper with decorator which measures time of the calculation and shows the current status
# --------------------------------------------------------------------------------------------


def wrap(pre, post):
    #     """ Wrapper """
    def decorate(func):
        #     """ Decorator """
        def call(*args, **kwargs):
            global interrupted
            interrupted = False
            start = time.perf_counter()
            thread = threading.Thread(
                target=work, args=(general_controls['progress'],))
            thread.start()
            pre(func, is_busy=True)
            result = func(*args, **kwargs)
            elapsed = time.perf_counter() - start
            interrupted = True
            post(func, elapsed, is_busy=False)

            return result
        return call
    return decorate


# ------------------------------------------------------------------------------------
# Helper functions for wrapped objects
# ------------------------------------------------------------------------------------

def entering(func, *args, **attributes):
    logger.info(f"Starting Function: {func.__name__}!")


def exiting(func, elapsed, *args, **attributes):
    logger.info(
        f"Run Function: {func.__name__} It took {elapsed:.2f} seconds!")


def isBusy(is_busy):
    return "red" if is_busy else "green"

# -----------------------------------------------------------------------------------------------------
# helper date function to convert a datetime.date format into a date string format
# -----------------------------------------------------------------------------------------------------


def convert_dt_to_string(date, format="%Y-%m-%d"):
    try:
        return date.strftime("%Y-%m-%d")
    except TypeError:
        logger.error("TypeError: Not a type of a datetime.date!")

# -----------------------------------------------------------------------------------------------------
# helper date function to convert string dates format into a datetime[64], working on df
# -----------------------------------------------------------------------------------------------------


def convert_with_format(df):
    return pd.to_datetime(df, format='%Y-%m-%d')


def convert_str_to_date(x, format='%Y-%m-%d'):
    return datetime.strptime(x, format).date() if isinstance(x, str) else x

# ---------------------------------------------------------------------------------------------------------------------
# These are the functions which are used for computing the values for standard periods and broken dates
# - get_accrued_days() End date - Start date as day difference
# - get_year_fraction() AccruedDays / Year Basis
# - get_realized_rate_lagged() LaggedAccruedPercent / YearFraction
# - get_realized_rate() AccruedPercent / YearFraction
# - get_realized_amount() RealizedRate * amount
# ----------------------------------------------------------------------------------------------------------------------


def get_accrued_days(end, start):
    return (convert_str_to_date(end) - convert_str_to_date(start)).days


def get_year_fraction(accrued_days, year_basis, rounding=10):
    try:
        result = round(accrued_days / year_basis, rounding)
    except ZeroDivisionError as error:
        result = 0.0
    return result


def get_realized_rate(accrued, yf, rounding):

    try:
        result = round(accrued / yf, int(rounding))
    except ZeroDivisionError as error:
        result = 0.0
    return result


def get_realized_amount(rate, amt, rounding):

    try:
        result = round(rate * amt / 100, rounding)
    except:
        logger.warning(f' Wrong values for {rate}, {amt} ,{rounding}')
        result = 0.0
    return result


def create_hookup(func: Callable, widgets: dict, tag: Any):
    def has_changed(value):
        func(tag, value)
    for w in widgets.values():
        w.observe(has_changed, 'value')

<h2> Refinitiv Data Libraries section </h2>

In [4]:
# ------------------------------------------------------------------------------------
# create_add_period_element generates a JSON document that can be sent
# to the add_period end-point of Instrument Pricing Analytics. the document is created
# in such a way that the period is added backward from an end date
# By default if the period is 1D/ON the moving convention used is the PreviousBusinessDay
# to take holidays and weekends into consideration and avoid end_date and start_date overlap
# ------------------------------------------------------------------------------------

def create_add_period_element(end_date, period, calendar):
    # ModifiedFollowing, PreviousBusinessDay, NextBusinessDay, NoMoving, BBSWModifiedFollowing
    dateMovingConvention = 'ModifiedFollowing'
    if period == '1D':
        dateMovingConvention = 'PreviousBusinessDay'

    period_element = rd.dates_and_calendars.add_periods(
        start_date=end_date,
        period="-" + period,
        calendars=[calendar],
        date_moving_convention=dateMovingConvention,
        end_of_month_convention="Last"
    )

    return {'Start': str(period_element), 'Tenor': period}


def generate_dates(end_date, index, periods):
    # ---------------------------------------------------------------------------
    # generate_dates computes dates from a refernce date, a list of tenors,
    # an index and a calendar.
    # the calcualtion is done by sending to the tbe add-periods end-point
    # a request built with the help of the create_add_period_element function
    # ---------------------------------------------------------------------------
    calendar = metadata["term_rate_assumption"][index]["calendar"]

    response = [create_add_period_element(
        end_date, period_code, calendar) for period_code in periods]

    df = pd.DataFrame(data=response)
    df['End'] = end_date
    df = df[['Tenor', 'Start', 'End']]

    return df


def add_period(end_date, period_code):
    # ---------------------------------------------------------------------------
    # add_period computes date from a refernce date, using specified lag tenor,
    # an index and a calendar.
    # the calcualtion is done by sending to the tbe add-periods end-point
    # a request built with the help of the create_add_period_element function
    # ---------------------------------------------------------------------------
    index = general_controls['index'].value
    calendar = metadata["term_rate_assumption"][index]["calendar"]

    response = create_add_period_element(end_date, period_code, calendar)

    return response['Start']


def backward_shift(end_date, lag):
    # --------------------------------------------------------------------------
    # Method used for the calculation of backward shifted date
    # it is calling the add_period method
    # --------------------------------------------------------------------------
    shifted_date = add_period(end_date, f'{lag}WD')
    return shifted_date

# ------------------------------------------------------------------------------------
# create_swap_element generates a swap instrument JSON definition that can be sent to
# the financial-contract end-point of Instrument Pricing Analytics
# ------------------------------------------------------------------------------------


def create_swap_element(start_date, end_date, amount, spread, day_count, currency, index, lag, override_lag=None, **kwargs):

    swap_definition = swap.Definition(
        instrument_tag=start_date,
        start_date=start_date,
        tenor="2Y",
        legs=[
            swap.LegDefinition(
                leg_tag="Fixed",
                interest_type="Fixed",
                direction="Paid",
                notional_ccy=currency,
                interest_payment_frequency="Zero",
                notional_amount=amount,
                interest_calculation_method=day_count,
                fixed_rate_percent=1.0),
            swap.LegDefinition(
                leg_tag="Float",
                direction="Received",
                notional_ccy=currency,
                interest_payment_frequency="Zero",
                notional_amount=amount,
                interest_type="Float",
                spread_bp=spread,
                interest_calculation_method=day_count,
                index_tenor="ON",
                index_name=index,
                index_fixing_lag=override_lag if override_lag is not None else lag,
                index_compounding_method="Compounded"),
        ],
        pricing_parameters=swap.PricingParameters(
            valuation_date=end_date),
    )

    logger.debug(swap_definition)
    return swap_definition

# -------------------------------------------------------------------------------------------------
# get_accrued_from_swaps makes a call to the financial-instrument end-point for the list of swaps
# created by generate_standard_request or/and generate_custom_request methods and passes the result
# to back to main loop. It digests a schedule(key) and swaps wrapped in the request
# --------------------------------------------------------------------------------------------------


async def get_accrued_from_swaps(schedule, swaps):

    logger.debug(swaps)

    swap = rd.content.ipa.financial_contracts.Definitions(
        universe=swaps,
        fields=["InstrumentTag", "LegTag",
                "AccruedPercent", "ErrorCode", "ErrorMessage"]
    ).get_data()

    try:
        headers_name = [h['name'] for h in swap.data.raw['headers']]
        df = pd.DataFrame(data=swap.data.raw['data'], columns=headers_name)
    except KeyError as e:
        logger.error(
            f'While getting results from IPA - For {schedule} - {swap.status} - {e}')
        return None
    else:
        # Get Floating rate Accrued Percentage
        mask = df['LegTag'] == 'Float'
        df = df[mask]
    logger.debug(df['AccruedPercent'])
    return df['AccruedPercent'].to_list()

<h2> Model </h2>

In [5]:

def fetch_standard_results(rates, schedule, year_basis, amount, rounding, **kwargs):

    # ---------------------------------------------------------------------------------------------------------------------
    # fetch_standard_results() is returning the data frame with the final results for standard periods.
    # AccruedPercent & LaggedAccrued are computed and are used for computing the final
    # RealizedRate & LaggedRealizedRate. The helper functions are called accordingly to calculate all values
    # If the 'standard_columns' list in default_data is changed or extended this method needs to be amended
    # ----------------------------------------------------------------------------------------------------------------------
    columns = default_data['standard_columns']
    output_df = schedule.copy()
    if not "AccruedDays" in output_df:
        output_df['AccruedDays'] = output_df.apply(
            lambda x: get_accrued_days(x['End'], x['Start']), axis=1)
        output_df['YearFraction'] = output_df.apply(
            lambda x: get_year_fraction(x['AccruedDays'], year_basis), axis=1)
    if 'standard' in rates:
        output_df['AccruedPercent'] = [
            round(rate, 10) for rate in rates['standard']]
        output_df['RealizedRate'] = output_df.apply(lambda x: get_realized_rate(
            x['AccruedPercent'], x['YearFraction'], rounding), axis=1)
        output_df['RealizedAmount'] = output_df.apply(
            lambda x: get_realized_amount(x['RealizedRate'], amount, rounding), axis=1)
    if 'standard_lag' in rates:
        output_df['LaggedAccrued'] = [
            round(rate, 10) for rate in rates['standard_lag']]
        output_df['LaggedRealizedRate'] = output_df.apply(lambda x: get_realized_rate(
            x['LaggedAccrued'], x['YearFraction'], rounding), axis=1)
    else:
        output_df['LaggedAccrued'] = [
            round(rate, 10) for rate in rates['standard']]
        output_df['LaggedRealizedRate'] = output_df.apply(lambda x: get_realized_rate(
            x['LaggedAccrued'], x['YearFraction'], rounding), axis=1)
    if 'backward_standard' in rates:
        output_df['BackwardAccrued'] = [
            round(rate, 10) for rate in rates['backward_standard']]
        output_df['BackwardShift'] = output_df.apply(lambda x: get_realized_rate(
            x['BackwardAccrued'], x['YearFraction'], rounding), axis=1)
    else:
        output_df['BackwardAccrued'] = [
            round(rate, 10) for rate in rates['standard']]
        output_df['BackwardShift'] = output_df.apply(lambda x: get_realized_rate(
            x['BackwardAccrued'], x['YearFraction'], rounding), axis=1)

    return output_df[columns]


def fetch_custom_results(rates, schedule, year_basis, amount, rounding, **kwargs):
    # ---------------------------------------------------------------------------------------------------------------------
    # fetch_standard_results() is returning the data frame with the final results for broken periods.
    # AccruedPercent is computed and are used for computing the final
    # RealizedRate. The helper functions are called accordingly to calculate all values
    # If the 'custom_columns' list in default_data is changed or extended this method needs to be amended
    # ----------------------------------------------------------------------------------------------------------------------

    columns = default_data['custom_columns']
    output_df = schedule.copy()
    output_df['AccruedDays'] = output_df.apply(
        lambda x: get_accrued_days(x['End'], x['Start']), axis=1)
    output_df['AccruedPercent'] = [round(rate, 10) for rate in rates['custom']]
    output_df['YearFraction'] = output_df.apply(
        lambda x: get_year_fraction(x['AccruedDays'], year_basis), axis=1)
    output_df['RealizedRate'] = output_df.apply(lambda x: get_realized_rate(
        x['AccruedPercent'], x['YearFraction'], rounding), axis=1)
    output_df['RealizedAmount'] = output_df.apply(
        lambda x: get_realized_amount(x['RealizedRate'], amount, rounding), axis=1)

    return output_df[columns]


def generate_requests(schedules, **kwargs):
    # ------------------------------------------------------------------------------------------------------------------------
    # generate_request method returns dictionary of requests for each schedule. The request consists of a swap list for each of realized
    # periods in schedule.
    # ------------------------------------------------------------------------------------------------------------------------
    standard_input_data = get_standard_input_data()
    custom_input_data = get_custom_input_data()
    input_data_dict = {'custom': custom_input_data, 'backward_custom': custom_input_data, 'custom_lag': custom_input_data,
                       'standard': standard_input_data, 'standard_lag': standard_input_data, 'backward_standard': standard_input_data}
    lag_exceptions = ['backward_custom', 'standard', 'backward_standard']
    requests = {}
    for key, schedule in schedules.items():
        if key in lag_exceptions:
            requests[key] = tuple(create_swap_element(swap["Start"], swap["End"], **
                                  input_data_dict[key], override_lag=0) for idx, swap in schedules[key].iterrows())
        else:
            requests[key] = tuple(create_swap_element(
                swap["Start"], swap["End"], **input_data_dict[key]) for idx, swap in schedules[key].iterrows())

    return requests


def get_standard_input_data():
    # ---------------------------------------------------------------------------------------------------------------------
    # get_standard_input_data returns the input data necessary
    # to compute the realized rate given the current state of standard controls:
    # - the nominal amount, the index fixing lag, the spread applied to the index rate, the index rounding
    # - the index name, and currency
    # - the daycount basis, year basis, and calendar
    # ----------------------------------------------------------------------------------------------------------------------
    index = general_controls['index'].value
    input_data = {
        'amount': float(standard_controls['amount'].value),
        'lag': standard_controls['lag'].value,
        'spread': standard_controls['spread'].value,
        'rounding': standard_controls['rounding'].value,
        'index': index,
        'currency': term_rate_assumption[index]['currency'],
        'day_count': standard_controls['day_count'].value,
        'calendar': term_rate_assumption[index]['calendar'],
        'year_basis': standard_controls['year_basis'].value
    }
    return input_data


def get_custom_input_data():
    # ---------------------------------------------------------------------------------------------------------------------
    # get_custom_input_data returns the input data necessary
    # to compute the realized rate given the current state of custom controls:
    # - the nominal amount, the index fixing lag, the spread applied to the index rate, the index rounding
    # - the index name, and currency
    # - the daycount basis, year basis, and calendar
    # ----------------------------------------------------------------------------------------------------------------------
    index = general_controls['index'].value
    input_data = {
        'amount': float(custom_controls['amount'].value),
        'lag': custom_controls['lag'].value,
        'spread': custom_controls['spread'].value,
        'rounding': custom_controls['rounding'].value,
        'index': index,
        'currency': term_rate_assumption[index]['currency'],
        'day_count': custom_controls['day_count'].value,
        'calendar': term_rate_assumption[index]['calendar'],
        'year_basis': custom_controls['year_basis'].value
    }
    return input_data


async def gather_dict(tasks: dict):
    # ---------------------------------------------------------------------------------------------------------------------
    # We want to mark each schedule with a key and gather the results for each schedule to a dictionary
    # ----------------------------------------------------------------------------------------------------------------------
    async def mark(key, coro):
        return key, await coro

    return {
        key: result
        for key, result in await asyncio.gather(
            *(mark(key, coro) for key, coro in tasks.items())
        )
    }


async def runner(tasks):
    # -------------------------------------------------------------------------------------------------------------------------------------------
    # this is the asynchronus loop which gets single or multiple requests from calculate_realized_rates method and runs them concurently
    # The results are gathered and cleaned since in the response from financial-contracts endpoint we get both fixed and float legs of the swap.
    # We are interested only in float leg. Outputs are converted to DataFrames.
    # -------------------------------------------------------------------------------------------------------------------------------------------
    # return asyncio.run(gather_dict(tasks))
    return await gather_dict(tasks)


async def calculate_realized_rates(schedules, request=None, override=False):
    # --------------------------------------------------------------------------------------------------------------------------------
    # calculate_realized_rates method returns fetched results recieved from the financial-contracts endpoint.
    # the mandatory input is a dictionary of dataframes. Acceptable keys are:
    # -'standard', 'standard_lag', 'backward_standard','custom', 'custom_lag', 'backward_custom'
    # Dataframes must have Start and End columns for period dates and may also carry additional data which we don't want to update
    # PLease see the fetch_standard_results for the logic behind.
    # Steps:
    # - For each dataframe provided it generates a request consisting of swaps for each period in dataframe. It is done by
    # calling generate_request method.
    # - Prepared requests are sent to the asyncio loop and
    # - calculated accrued percent lists are gathered for each dataframe and sent for fetching
    # --------------------------------------------------------------------------------------------------------------------------------
    standard_input_data = get_standard_input_data()
    custom_input_data = get_custom_input_data()
    # print(schedules)
    task_schedules = generate_requests(schedules=schedules)
    tasks = {schedule: get_accrued_from_swaps(
        schedule, swaps) for schedule, swaps in task_schedules.items()}

    result = await runner(tasks)

    for schedule, rates in result.items():
        if not rates:
            logger.warning(
                f'For {schedule} the accrued rates are missing, replacing with 0s')
            rates = [0] * schedules[schedule].shape[0]
            result[schedule] = rates

    output = {}
    for key in ('standard', 'standard_lag', 'backward_standard'):
        if key in result:
            output['standard'] = fetch_standard_results(
                result, schedules[key], **standard_input_data)
            break

    for key in ('custom', 'custom_lag', 'backward_custom'):
        if key in result:
            output['custom'] = fetch_custom_results(
                result, schedules[key], **custom_input_data)
            break

    return output

<h2> Ipywidget section (setting up controls) </h2>
<ul>
<li> general_controls: index, progress : controls which are governing both Standard and Custom period views
<li> standard_controls: consists of controls which are common to standard period view
<li> custom_controls: consists of controls which are common to custom period view
<li> labels_dict: consists of labels for the Gridspeclayout which is the container for presenting custom period data
</ul>

In [6]:
general_style = {'description_width': '150px'}
general_layout = {'width': 'auto'}
row_style = {'description_width': '150px'}
row_layout = {'width': '100px'}
box_layout2 = Layout(width='150%', display='inline-flex',
                     align_items='flex-start', flex_flow='row')
box_layout = Layout(width='100%', display='inline-flex',
                    flex_flow='row', align_items='flex-start')


general_controls = dict(
    index=widgets.Dropdown(options=list(metadata['term_rate_assumption'].keys()),
                           value='SONIA', description='Index:', disabled=False,
                           continuous_update=False, layout={'width': '200px', 'hight': '40px'}, style={'description_width': 'initial'}),
    progress=widgets.FloatProgress(
        value=0.0, min=0.0, max=1.0, description='Loading:')
)

standard_controls = dict(
    calendar=widgets.Text(placeholder='', description='Calendar:',
                          disabled=True, layout=general_layout, style=general_style),
    year_basis=widgets.FloatText(placeholder='', description='Year basis:',
                                 disabled=True, layout=general_layout, style=general_style),
    lag=widgets.Dropdown(placeholder='', options=default_data['lags'], description='Fixing lag:', value=int(default_data['lag']), disabled=False,
                         continuous_update=True, layout={'width': '200px'}, style={'description_width': 'initial'}),
    spread=widgets.IntText(placeholder='', description='Spread:', value=int(default_data['spread']), disabled=False,
                           continuous_update=False, layout=general_layout, style={'description_width': 'initial'}),
    rounding=widgets.IntText(placeholder='', description='Rate rounding (dec.):', value=int(default_data['rounding']),
                             disabled=True, continuous_update=False, layout=general_layout, style=general_style),
    amount=widgets.FloatText(placeholder='', description='Amount:', value=float(default_data['amount']), disabled=False,
                             continuous_update=False, layout=general_layout, style=general_style),
    calculate=widgets.Button(description='Calculate', disabled=False, button_style='',
                             tooltip='Click me to update the data', icon='check', layout=general_layout, style=general_style),
    end_date=widgets.DatePicker(disabled=False, description='End:', value=default_data['date'], continuous_update=False,
                                layout=general_layout, style={'description_width': 'initial'}),
    day_count=widgets.Text(disabled=False, description='Day Count:', value=default_data['day_count'],
                           continuous_update=False, layout=general_layout, style=general_style),
    export_to_excel=Button(description='Export', icon='fa-file-excel-o',
                           tooltip='Export view to the current folder')
)

custom_controls = dict(

    calendar=widgets.Text(placeholder='', description='Calendar:',
                          disabled=True, layout=general_layout, style=general_style),
    add_row=widgets.Button(description='Add Row'),
    delete_row=widgets.Button(description='Delete Row'),
    year_basis=widgets.Dropdown(placeholder='', options=list(default_data['year_basis']), value=int(term_rate_assumption['SONIA']['year_basis']), description='Year basis:', disabled=False,
                                layout=general_layout, style=general_style),
    lag=widgets.Dropdown(placeholder='', options=default_data['lags'], description='Fixing lag:', value=int(default_data['lag']), disabled=False,
                         continuous_update=True, layout=general_layout, style={'description_width': 'initial'}),
    spread=widgets.IntText(placeholder='', description='Spread:', value=int(default_data['spread']), disabled=False, continuous_update=False,
                           layout=general_layout, style={'description_width': 'initial'}),
    rounding=widgets.IntText(placeholder='', description='Rate rounding (dec.):', value=int(default_data['rounding']),
                             disabled=False, continuous_update=False, layout=general_layout, style=general_style),
    amount=widgets.FloatText(placeholder='', description='Amount:', value=float(default_data['amount']), disabled=False,
                             continuous_update=True, layout=general_layout, style={'description_width': 'initial'}),
    calculate=widgets.Button(description='Calculate', disabled=False, button_style='',
                             tooltip='Click me to update the data', icon='check', layout=general_layout, style=general_style),
    day_count=widgets.Text(disabled=False, description='Day Count:', value=default_data['day_count'], continuous_update=False,
                           layout=general_layout, style=general_style),
    export_to_excel=widgets.Button(
        description='Export', icon='fa-file-excel-o', tooltip='Export view to the current folder')
)


labels_dict = dict(
    row_id=Button(description='Id', layout=Layout(width='40px')),
    tenor=Button(description='Tenor', layout=Layout(width='50px')),
    start=Button(description='Start Date', layout=Layout(width='auto'),),
    end=Button(description='End Date', layout=Layout(width='auto'),),
    accrued_days=Button(description='AccruedDays', layout=general_layout,),
    accrued_percent=Button(description='AccruedPercent',
                           layout=general_layout,),
    year_fraction=Button(description='YearFraction', layout=general_layout,),
    realized_rate=Button(description='RealizedRate', layout=general_layout,),
    realized_amt=Button(description='RealizedAmt', layout=general_layout,)
)


# -----------------------------------------------------------------------------
# the index dropdown (index_widget) controls the values displayed in
# the text box calendar, yearbasis, rounding
# -----------------------------------------------------------------------------

def get_calendar(value):
    return metadata['term_rate_assumption'][value]['calendar']


def get_yearbasis(value):
    return float(metadata['term_rate_assumption'][value]['year_basis'])


def get_rounding(value):
    return int(metadata['term_rate_assumption'][value]['rounding'])


def get_lag(value):
    return int(metadata['term_rate_assumption'][value]['lag'])


directional_link((general_controls['index'], 'value'),
                 (standard_controls['calendar'], 'value'), get_calendar)
directional_link((general_controls['index'], 'value'),
                 (standard_controls['year_basis'], 'value'), get_yearbasis)
directional_link((general_controls['index'], 'value'),
                 (standard_controls['rounding'], 'value'), get_rounding)

directional_link((general_controls['index'], 'value'),
                 (custom_controls['calendar'], 'value'), get_calendar)
directional_link((general_controls['index'], 'value'),
                 (custom_controls['year_basis'], 'value'), get_yearbasis)
directional_link((general_controls['index'], 'value'),
                 (custom_controls['rounding'], 'value'), get_rounding)

/var/folders/hd/5zrct37502z2kq6ml9_65cn80000gn/T/ipykernel_4917/1582507746.py:38: DeprecationWarning: icons names no longer need 'fa-', just use the class names themselves (for example, 'gear spin' instead of 'fa-gear fa-spin')
  export_to_excel=Button(description='Export', icon='fa-file-excel-o',
/var/folders/hd/5zrct37502z2kq6ml9_65cn80000gn/T/ipykernel_4917/1582507746.py:62: DeprecationWarning: icons names no longer need 'fa-', just use the class names themselves (for example, 'gear spin' instead of 'fa-gear fa-spin')
  export_to_excel=widgets.Button(


<h2> Setting up the interactions between the gui (view) and the controller </h2>
<p>  in this section you may find methods used for creating and manipulating general_controls,
     standard_controls and custom_controls </p>

In [7]:


# ------------------------------------------------------------------------------------------------------------
# The calcualted realized term rates and a few intermediary calcualtions are displayed
# in a ipysheet object.
# ------------------------------------------------------------------------------------------------------------


async def initiate_spreadsheet():
    # ---------------------------------------------------------------------------------------------------------
    # This function is called when the calculator is initiated.
    # It creates a sheet_template using provided data DataFrame.
    # The number of rows depends on the number of tenors defined in
    # the default value of the tenor list column (see metadata object).
    # The number of columns depends on the columns of input dataframe (see metadata object) and by default is:
    # - Tenor, Start, End, AccruedDays, AccruedPercent, YearFraction, RealizedRate and LaggedRealizedRate
    # ---------------------------------------------------------------------------------------------------------
    schedule_standard = generate_dates(index=general_controls['index'].value, end_date=standard_controls['end_date'].value.strftime('%Y-%m-%d'),
                                       periods=default_data['periods'])

    schedules = {'standard': schedule_standard}
    standard_results = await calculate_realized_rates(schedules)

    num_format = '0.{}'.format('0'*int(default_data['rounding']))
    sheet_template = ipysheet.sheet(
        from_dataframe(standard_results['standard']))
    sheet_template.column_width = [30, 40, 40, 40, 40, 40, 40, 40]
    sheet_template.row_headers = False
    sheet_template.cells[3].numeric_format = '0'
    sheet_template.cells[4].numeric_format = num_format
    sheet_template.cells[5].numeric_format = num_format
    for x in range(0, len(sheet_template.cells)):
        sheet_template.cells[x].read_only = True

    for x in range(4, len(sheet_template.cells)):
        sheet_template.cells[x].numeric_format = num_format
    return sheet_template


def update_spreadsheet(new_data_df):
    # -------------------------------------------------------------------------------------------------
    # This is a helper function which populates the existing sheet with the new data.
    # New data is provided in the data frame with the same number of columns as initialy created sheet
    # -------------------------------------------------------------------------------------------------
    for idx, column in enumerate(sht.cells):
        column.value = new_data_df.iloc[:, idx].to_list()

    return

def calculate_spreadsheet(*args, **kwargs):
    asyncio.ensure_future(calculate_spreadsheet_async(*args, **kwargs))

@wrap(entering, exiting)
async def calculate_spreadsheet_async(*args, **kwargs):
    # -----------------------------------------------------------------------------
    # each time the user changes one of the value of end_date widget,
    # the realized rates are recomputed and passed as a dataframe to update_spreadsheet().
    # When the lag  =  =  0 then we can populate LaggedRealizedRate and BackwardShift with RealizedRate
    # since in this scenario all rates should be the same.
    # -----------------------------------------------------------------------------
    if standard_controls['lag'].value == 0:
        schedule_standard = generate_dates(index=general_controls['index'].value, end_date=standard_controls['end_date'].value.strftime('%Y-%m-%d'),
                                           periods=default_data['periods'])

        schedules = {'standard': schedule_standard}
    else:
        schedule_standard = generate_dates(index=general_controls['index'].value, end_date=standard_controls['end_date'].value.strftime('%Y-%m-%d'),
                                           periods=default_data['periods'])
        backward_end_date = backward_shift(standard_controls['end_date'].value.strftime(
            '%Y-%m-%d'), standard_controls['lag'].value)
        schedule_standard_back = generate_dates(
            index=general_controls['index'].value, end_date=backward_end_date, periods=default_data['periods'])

        schedules = {'standard': schedule_standard, 'standard_lag': schedule_standard,
                     'backward_standard': schedule_standard_back}

    standard_results = await calculate_realized_rates(schedules)
    update_spreadsheet(standard_results['standard'])
    return

def on_spread_change(spread):
    asyncio.ensure_future(on_spread_change_async(spread))

@wrap(entering, exiting)
async def on_spread_change_async(spread):
    # --------------------------------------------------------------------------------------------------------
    # on_spread_change triggers the recalculation of RealizedRates,LaggedRealizedRates and BackwardShift
    # for standard periods.
    # When the lag  =  =  0 then we can populate LaggedRealizedRate and BackwardShift with RealizedRate
    # since in this scenario all rates should be the same.
    # --------------------------------------------------------------------------------------------------------
    if standard_controls['lag'].value == 0:
        schedule_standard = generate_dates(index=general_controls['index'].value, end_date=standard_controls['end_date'].value.strftime('%Y-%m-%d'),
                                           periods=default_data['periods'])

        schedules = {'standard': schedule_standard}
    else:
        schedule_standard = generate_dates(index=general_controls['index'].value, end_date=standard_controls['end_date'].value.strftime('%Y-%m-%d'),
                                           periods=default_data['periods'])
        backward_end_date = backward_shift(standard_controls['end_date'].value.strftime(
            '%Y-%m-%d'), standard_controls['lag'].value)
        schedule_standard_back = generate_dates(
            index=general_controls['index'].value, end_date=backward_end_date, periods=default_data['periods'])

        schedules = {'standard': schedule_standard, 'standard_lag': schedule_standard,
                     'backward_standard': schedule_standard_back}

    standard_results = await calculate_realized_rates(schedules)
    update_spreadsheet(standard_results['standard'])
    return

def on_fixing_change(lag):
    asyncio.ensure_future(on_fixing_change_async(lag))

@wrap(entering, exiting)
async def on_fixing_change_async(lag):
    # -----------------------------------------------------------------------------
    # on_finxing_change triggers the recalculation of LaggedRealizedRates for standard periods
    # When the lag  =  =  0 then we can populate LaggedRealizedRate and BackwardShift with RealizedRate
    # since in this scenario all rates should be the same.
    # -----------------------------------------------------------------------------
    global sht
    df = ipysheet.to_dataframe(sht)
    if lag.new == 0:
        for idx, column_header in enumerate(sht.column_headers):
            if column_header in ['LaggedRealizedRate', 'BackwardShift']:
                sht.cells[idx].value = df['RealizedRate'].to_list()
        return
    schedule_standard = df[['Tenor', 'Start', 'End', 'AccruedDays',
                            'YearFraction', 'AccruedPercent', 'RealizedRate']]
    schedules = {'standard_lag': schedule_standard}
    backward_end_date = backward_shift(
        standard_controls['end_date'].value.strftime('%Y-%m-%d'), lag.new)
    schedule_standard_back = generate_dates(
        index=general_controls['index'].value, end_date=backward_end_date, periods=default_data['periods'])
    schedules = {'standard_lag': schedule_standard,
                 'backward_standard': schedule_standard_back}

    result = await calculate_realized_rates(schedules)
    rf = result['standard']
    for idx, column_header in enumerate(sht.column_headers):
        if column_header in ['LaggedRealizedRate', 'BackwardShift']:
            sht.cells[idx].value = rf[column_header].to_list()
    return


# -------------------------------------------------------------------------------------------------
# For custom view broken-periods the calcualted realized term rates and other intermediary calcualtions are displayed
# in separate ipywidget objects in each row for each broken period
# -------------------------------------------------------------------------------------------------

def get_custom_dates():
    # -------------------------------------------------------------------------------------------------
    # This is a helper function which creates a date schedule from available custom rows
    # -------------------------------------------------------------------------------------------------
    dates = [[row['tenor'].value, convert_dt_to_string(row['start'].value), convert_dt_to_string(
        row['end'].value)] for key, row in rows_custom.items()]
    schedule = pd.DataFrame(data=dates, columns=['Tenor', 'Start', 'End'])
    return schedule


def create_row(row_id, start_date, end_date):
    # ------------------------------------------------------------------------------------------------------------------
    # create_row method creates a row of widgets for the custom periods display
    # each created row is added to the rows_custom dict with a dedicated row_id
    # the start and end widgets are interactive and will trigger the recalculation by calling new_custom_row_data
    # ------------------------------------------------------------------------------------------------------------------
    row_widgets = dict(
        row_id=widgets.IntText(value=row_id, layout=Layout(
            width='40px'), disabled=True),
        tenor=widgets.Text(value='custom', disabled=False,
                           layout=Layout(width='50px')),
        start=widgets.DatePicker(disabled=False, value=convert_str_to_date(
            start_date), continuous_update=False, layout=Layout(width='auto')),
        end=widgets.DatePicker(disabled=False, value=convert_str_to_date(
            end_date), continuous_update=False, layout=Layout(width='auto')),
        accrued_days=widgets.IntText(
            value=0.0, disabled=False, layout=general_layout),
        accrued_percent=widgets.FloatText(
            value=0.0, disabled=False, layout=general_layout),
        year_fraction=widgets.FloatText(
            value=0.0, disabled=False, layout=general_layout),
        realized_rate=widgets.FloatText(
            value=0.0, disabled=False, layout=general_layout),
        realized_amt=widgets.FloatText(
            value=0.0, disabled=False, layout=general_layout)
    )

    rows_custom[row_id] = row_widgets
    create_hookup(func=new_custom_row_data, widgets={
                  'start_v': rows_custom[row_id]['start'], 'end_v': rows_custom[row_id]['end']}, tag=row_id)
    return


def create_custom_rows(start, end, number_of_rows=1):
    # -------------------------------------------------------------------------------------------------
    # This is a helper function which creates rows using incoming data frame. The number_of_rows argument
    # specifies how many rows we want to add. In case there are rows already the function is not executed
    # -------------------------------------------------------------------------------------------------
    global rows_custom
    for idx in range(0, number_of_rows):
        create_row(idx, start, end)
        rows_custom_ui.append(widgets.HBox(
            list(rows_custom[idx].values()), layout=box_layout))

    return


def update_custom_row(new_data_df, idx):
    # --------------------------------------------------------------------------------------
    # populates the data calculated using new start or end dates to the specific row by idx
    # ---------------------------------------------------------------------------------------

    rows_custom[idx]['accrued_days'].value = new_data_df['AccruedDays'][0]
    rows_custom[idx]['accrued_percent'].value = new_data_df['AccruedPercent'][0]
    rows_custom[idx]['year_fraction'].value = new_data_df['YearFraction'][0]
    rows_custom[idx]['realized_rate'].value = new_data_df['RealizedRate'][0]
    rows_custom[idx]['realized_amt'].value = new_data_df['RealizedAmount'][0]

    return


def update_custom_rows(data_df):
    # --------------------------------------------------------------------------------------
    # populates the data calculated using new start or end dates to all rows
    # ---------------------------------------------------------------------------------------
    for idx, item in rows_custom.items():
        rows_custom[idx]['accrued_days'].value = data_df['AccruedDays'][idx]
        rows_custom[idx]['accrued_percent'].value = data_df['AccruedPercent'][idx]
        rows_custom[idx]['year_fraction'].value = data_df['YearFraction'][idx]
        rows_custom[idx]['realized_rate'].value = data_df['RealizedRate'][idx]
        rows_custom[idx]['realized_amt'].value = data_df['RealizedAmount'][idx]

def new_custom_row_data(tag, value):
    asyncio.ensure_future(new_custom_row_data_async(tag, value))

async def new_custom_row_data_async(tag, value):
    # -----------------------------------------------------------------------------
    # each time the user changes start or end date in a specific row,
    # the realized rates are recomputed and passed as a dataframe to update_custom_row()
    # which populates the data to this row
    # -----------------------------------------------------------------------------
    schedule_custom = pd.DataFrame(dict(Tenor='custom', Start=rows_custom[tag]['start'].value.strftime(
        '%Y-%m-%d'), End=rows_custom[tag]['end'].value.strftime('%Y-%m-%d')), index=[0])
    schedules = {'custom': schedule_custom}
    custom_results = await calculate_realized_rates(schedules)
    update_custom_row(custom_results['custom'], tag)
    return

def update_custom_rows_on_event(*args, **kwargs):
    asyncio.ensure_future(update_custom_rows_on_event_async(*args, **kwargs))


@wrap(entering, exiting)
async def update_custom_rows_on_event_async(*args, **kwargs):
    # -----------------------------------------------------------------------------
    # each time the user changes lag or spread all custom periods are recalculated,
    # the realized rates are recomputed and passed as a dataframe to update_custom_row()
    # -----------------------------------------------------------------------------
    schedule_custom = get_custom_dates()
    schedules = {'custom': schedule_custom}
    custom_results = await calculate_realized_rates(schedules)
    update_custom_rows(custom_results['custom'])
    return


# -----------------------------------------------------------------------------------
# on change: amount, year_basis and rounding no api call is required hence
# the calculations are made manualy using a specific functions
# ------------------------------------------------------------------------------------

def update_custom_amount(amt):
    for key, row in rows_custom.items():
        row['realized_amt'].value = get_realized_amount(
            row['realized_rate'].value, amt.new, custom_controls['rounding'].value)

    return


def update_custom_year_basis(year_basis):
    for key, row in rows_custom.items():
        row['year_fraction'].value = get_year_fraction(
            row['accrued_days'].value, year_basis.new, 6)
        row['realized_rate'].value = get_realized_rate(
            row['accrued_percent'].value, row['year_fraction'].value, custom_controls['rounding'].value)
        row['realized_amt'].value = get_realized_amount(
            row['realized_rate'].value, custom_controls['amount'].value, custom_controls['rounding'].value)
    return


def update_custom_rounding(rounding):
    for key, row in rows_custom.items():
        row['realized_rate'].value = get_realized_rate(
            row['accrued_percent'].value, row['year_fraction'].value, rounding.new)
        row['realized_amt'].value = get_realized_amount(
            row['realized_rate'].value, custom_controls['amount'].value, rounding.new)
    return


def on_index_change(index):
    asyncio.ensure_future(on_index_change_async(index))


@wrap(entering, exiting)
async def on_index_change_async(index):
    # -----------------------------------------------------------------------------
    # on_index_change triggers the recalculation of standard periods and
    # custom periods using the current settings
    # -----------------------------------------------------------------------------

    if standard_controls['lag'].value == 0:
        schedule_standard = generate_dates(index=general_controls['index'].value, end_date=standard_controls['end_date'].value.strftime('%Y-%m-%d'),
                                           periods=default_data['periods'])
        schedule_custom = get_custom_dates()
        schedules = {'custom': schedule_custom, 'standard': schedule_standard}
    else:

        schedule_standard = generate_dates(index=general_controls['index'].value, end_date=standard_controls['end_date'].value.strftime(
            '%Y-%m-%d'), periods=default_data['periods'])

        schedule_standard_back = generate_dates(index=general_controls['index'].value,
                                                end_date=backward_shift(standard_controls['end_date'].value.strftime(
                                                    '%Y-%m-%d'), standard_controls['lag'].value),
                                                periods=default_data['periods'])
        schedule_custom = get_custom_dates()
        schedules = {'custom': schedule_custom, 'standard': schedule_standard,
                     'standard_lag': schedule_standard, 'backward_standard': schedule_standard_back}
    results = await calculate_realized_rates(schedules)
    update_custom_rows(results['custom'])
    update_spreadsheet(results['standard'])
    return


def export_custom_to_excel(_):
    # ----------------------------------------------------------------------------------------
    # export custom periods to xslx to the root directory
    # ----------------------------------------------------------------------------------------
    data = []
    for key, row in rows_custom.items():
        data.append(
            [widget.value for idx, widget in enumerate(row.values()) if idx > 0])

    df = pd.DataFrame(data=data, columns=default_data['custom_columns'])
    df.to_excel(f"Custom_periods_{general_controls['index'].value}.xlsx")
    return


def export_standard_to_excel(_):
    # ----------------------------------------------------------------------------------------
    # export standard periods to xslx to the root directory
    # ----------------------------------------------------------------------------------------
    df = ipysheet.to_dataframe(sht)
    df.to_excel(f"Standard_periods_{general_controls['index'].value}.xlsx")
    return


# @wrap(entering, exiting)
def delete_row(_):
    # -----------------------------------------------------------------------------
    # This function is used for deleting new row to the custom view
    # new row is added together with an interactive hookup which
    # means whenever start or end dates are changed the new_custom_row_data() function
    # is called and the other data in the same row is updated
    # -----------------------------------------------------------------------------
    global grid_custom
    global number_of_columns
    global grid_row_offset
    global grid_column_offset
    row_id = len(rows_custom)-1
    if row_id <= 0:
        return

    rows_custom_ui.pop(row_id)
    rows_custom.pop(row_id)
    grid_custom = create_grid(
        grid_row_offset, grid_column_offset, number_of_columns)
    ui_custom_children = list(ui_custom.children)
    ui_custom_children[1] = grid_custom
    ui_custom.children = ui_custom_children
    return


@wrap(entering, exiting)
def add_new_row(_):
    # -----------------------------------------------------------------------------
    # This function is used for adding new row to the custom view
    # new row is added together with an interactive hookup which
    # means whenever start or end dates are changed the new_custom_row_data() function
    # is called and the other data in the same row is updated
    # -----------------------------------------------------------------------------
    new_row_id = len(rows_custom)
    previous_row_id = new_row_id-1
    create_row(new_row_id, rows_custom[previous_row_id]['start'].value,
               rows_custom[previous_row_id]['end'].value)

    rows_custom_ui.append(widgets.HBox(
        list(rows_custom[new_row_id].values()), layout=box_layout))
    new_custom_row_data(new_row_id, '')
    for idx, widget in enumerate(list(rows_custom[new_row_id].values())):
        if idx >= grid_column_offset:
            grid_custom[new_row_id+grid_row_offset,
                        idx-grid_column_offset] = widget

    return

# -----------------------------------------------------------------------------------
# In the custom view user can add a new row or delete last row in custom periods.
# New row is added with the same values as seen in the previous row which already exists
# ------------------------------------------------------------------------------------


custom_controls['add_row'].on_click(add_new_row)
custom_controls['delete_row'].on_click(delete_row)
standard_controls['export_to_excel'].on_click(export_standard_to_excel)
custom_controls['export_to_excel'].on_click(export_custom_to_excel)


# ---------------------------------------------------------------------------------------------
# general hookup is set on index, so each time user changes the index widget value
# both standard and custom views are updated separately.
# ---------------------------------------------------------------------------------------------

general_controls['index'].observe(on_index_change, 'value')

# ---------------------------------------------------------------------------------------------
# custom hookups are set for custom view only. The custom view gets updated
# each time when user is changing custom controls: lag, spread, year_basis, amount, rounding.
# Custom controls give more flexibility to change the settings of the computations.
# ---------------------------------------------------------------------------------------------

custom_controls['lag'].observe(update_custom_rows_on_event, 'value')
custom_controls['spread'].observe(update_custom_rows_on_event, 'value')
custom_controls['year_basis'].observe(update_custom_year_basis, 'value')
custom_controls['amount'].observe(update_custom_amount, 'value')
custom_controls['rounding'].observe(update_custom_rounding, 'value')

# ---------------------------------------------------------------------------------------------
# general hookups are set for standard view only. The custom view gets updated
# each time when user changes any of the standard controls: end_date, lag, spread
# ---------------------------------------------------------------------------------------------

standard_controls['end_date'].observe(calculate_spreadsheet, 'value')
standard_controls['lag'].observe(on_fixing_change, 'value')
standard_controls['spread'].observe(on_spread_change, 'value')

def create_grid(grid_row_offset, grid_column_offset, number_of_columns):
    # --------------------------------------------------------------------------------
    # create_grid returns the gridspeclayout which is NxM grid
    # Custom periods data are displayed in this grid
    # by default returned grid has 10 rows and number of columns depands on the
    # number of custom_columns defined in the default_data dict
    # --------------------------------------------------------------------------------

    # initialize grids separately for Standard and Cusom views
    grid_template = GridspecLayout(10, number_of_columns, layout={
                                   'width': '100%', 'grid_gap': '0px', 'overflow': 'auto'})

    # populat row 0 with custom_controls to each column starting from grid_column_offset

    grid_template[0, 0] = custom_controls['add_row']
    grid_template[0, 1] = custom_controls['delete_row']
    grid_template[0, 2] = custom_controls['export_to_excel']
    grid_template[0, 4] = custom_controls['spread']
    grid_template[0, 5] = custom_controls['lag']
    grid_template[0, 6] = custom_controls['amount']

    # populat row 1 with labels to each column starting from grid_column_offset
    for idx, label in enumerate(list(labels_dict.values())):
        if idx >= grid_column_offset:
            grid_template[1, idx-grid_column_offset] = label

    # populat rest of the grid with custom_widgets
    for row_idx, row in rows_custom.items():
        for idx, widget in enumerate(list(row.values())):
            if idx >= grid_column_offset:
                grid_template[row_idx+grid_row_offset,
                              idx-grid_column_offset] = widget

    return grid_template

<h2> Ipywidgets gui is defined in this section </h2>

In [8]:
start = time.perf_counter()


# --------------------------------------------------------------------------------------------------------------------
# To create custom rows we are calling create_custom_rows().
# Since we are passing 1 as number_of_rows argument only 1 row is created
# --------------------------------------------------------------------------------------------------------------------

create_custom_rows(start='2022-01-01',
                   end=default_data['date'], number_of_rows=1)
update_custom_rows_on_event()
sht = await initiate_spreadsheet()


#############################################################################################################################
# Ipywidgets gui is defined in this section
#############################################################################################################################

# title is using html inline style
title = widgets.HTML(
    value="""<h1 style = "padding: 20px; text-align:center; background-color:SteelBlue; color:white; font-size: 20px">RFR Realized Rate Calculator</h1>""")

# first and second row are occupied by custom controls and labels thus we are setting the grid_row_offset to 2 to start populating widgets from this row
grid_row_offset = 2

# to show labels only specific labels and widgets we need to set grid_column_offset
grid_column_offset = 2

# number of columns for the grid is depending on the nummber of custom_columns defined in the default_data dictionary
number_of_columns = len(default_data['custom_columns'])

grid_custom = create_grid(
    grid_row_offset, grid_column_offset, number_of_columns)

# test the layouts to align the widgets
box_layout_test = Layout(width='100%', display='inline-flex', flex_flow='row')
# box_layout_test2 = Layout(width = '100%',display = 'inline-flex',flex_flow = 'row',align_items = 'flex-start',margin_left = '50px')

# wrapping general widgets in vertical boxes

standard_controls_ui = HBox([standard_controls['end_date'], standard_controls['lag'],
                            standard_controls['spread'], standard_controls['export_to_excel']],)

general_controls_ui = Accordion(
    children=[HBox([general_controls['index']], layout=box_layout_test)])
general_controls_ui.set_title(0, 'General Settings')

additional_custom_controls = HBox(
    [custom_controls['year_basis'], custom_controls['rounding']],)

additional_settings = Accordion(
    children=[additional_custom_controls], selected_index=None)
additional_settings.set_title(0, 'Additional Settings')

# wrap grids for Standard and Custom dates tables in separate VBoxes
ui_standard = VBox([standard_controls_ui, sht])
ui_custom = VBox([additional_settings, grid_custom])

# we will put the tables in separate tabs
tabs = Tab(children=[ui_standard, ui_custom])

# name th tabs accordingly
tabs.set_title(0, 'Standard periods')
tabs.set_title(1, 'Custom periods')

display(title, general_controls['progress'], general_controls_ui, tabs)

end = time.perf_counter() - start
logger.debug(f"Program loaded in {end:0.2f} seconds.")

HTML(value='<h1 style = "padding: 20px; text-align:center; background-color:SteelBlue; color:white; font-size:…

FloatProgress(value=1.0, description='Loading:', layout=Layout(visibility='hidden'), max=1.0)

Accordion(children=(HBox(children=(Dropdown(description='Index:', layout=Layout(width='200px'), options=('SONI…

In [9]:
rd.close_session()